In [31]:
import pumpp
from pydub import AudioSegment
import os

sound1 = AudioSegment.from_mp3('/Users/yuwang/Desktop/MARL MIR/deepunet_vocal_sep/test/instrumental/000002.mp3')
sound2 = AudioSegment.from_mp3('/Users/yuwang/Desktop/MARL MIR/deepunet_vocal_sep/test/vocal/000002.mp3')

# mix sound2 with sound1, starting at 5000ms into sound1)
output = sound1.overlay(sound2, position=5000)

# save the result
output.export("000002.mp3", format="mp3")

<_io.BufferedRandom name='mixed_sounds.mp3'>

In [53]:
T = pumpp.feature.STFTMag(name='stft', sr=44100, hop_length=512, n_fft=1024, log=False, conv='tf')
P = pumpp.Pump(T)

In [54]:
d1 = P.transform(audio_f='/Users/yuwang/Desktop/MARL MIR/deepunet_vocal_sep/test/mix/000002.mp3')
d2 = P.transform(audio_f='/Users/yuwang/Desktop/MARL MIR/deepunet_vocal_sep/test/instrumental/000002.mp3')

In [57]:
print(d2)

{'stft/mag': array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]], dtype=float32)}


In [59]:
d1['output/mag'] = d2['stft/mag']
print(d1)

{'stft/mag': array([[[[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        ...,

        [[5.9859296e-03],
         [4.2274878e-03],
         [1.4071262e-03],
         ...,
         [3.3753444e-05],
         [7.2950344e-05],
         [9.1841459e-05]],

        [[9.3404458e-05],
         [9.2909147e-05],
         [9.1438174e-05],
         ...,
         [2.6506459e-07],
         [2.7223518e-07],
         [2.7538468e-07]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+

In [60]:
# Only make this object when building the sampler (during training)

fake_T = pumpp.feature.STFTMag(name='output', sr=16000, hop_length=512, n_fft=1024, log=False, conv='tf')

In [61]:
# Make a sampler using our real transformer and the fake one for output
s = pumpp.Sampler(10, 50, T, fake_T)

In [62]:
d1.keys()

dict_keys(['stft/mag', 'output/mag'])

In [63]:
# Grab a sample from the data dict
next(s(d1))

{'output/mag': array([[[[1.6450179e+00],
          [4.8006802e+00],
          [3.4997461e+00],
          ...,
          [9.6514312e-05],
          [1.1788395e-04],
          [1.9209813e-04]],
 
         [[3.1655073e-01],
          [4.2733164e+00],
          [6.6905303e+00],
          ...,
          [7.3983865e-05],
          [1.4083354e-04],
          [2.6871410e-04]],
 
         [[4.7237065e-02],
          [1.4078211e+00],
          [3.3308005e+00],
          ...,
          [3.5514011e-05],
          [9.3975425e-05],
          [9.1868722e-05]],
 
         ...,
 
         [[5.7875404e+01],
          [6.0384075e+01],
          [4.4865284e+01],
          ...,
          [1.3954945e-04],
          [9.6099160e-05],
          [7.6400494e-05]],
 
         [[1.7714729e+01],
          [1.6262331e+01],
          [1.2189764e+01],
          ...,
          [1.9382351e-04],
          [8.7081047e-05],
          [3.4329008e-05]],
 
         [[2.7223916e+00],
          [3.0878968e+00],
          [2.018